# 🔐 Cryptography Theory and Practice
## Securing Communication and Data Integrity in Modern IT Environments

**Module:** Cryptography Theory and Practice  
**Programme:** PGDip in Cybersecurity  
**Lecturer:** Dr Matt Lemon

---

This notebook covers the three main topics for CA2:
1. **IPSEC Implementation** - ESP and AH protocols
2. **AES Encryption** - Modes of operation and key management
3. **SHA Hashing** - Data integrity and algorithm comparison

Run each cell in order to see the examples in action!

---
# Part 1: IPSEC Implementation

## What is IPSEC?

**IPsec (Internet Protocol Security)** is a suite of protocols designed to secure IP communications by authenticating and encrypting each IP packet in a data stream.

### Key Components:

| Protocol | Purpose | What it Protects |
|----------|---------|------------------|
| **ESP** (Encapsulating Security Payload) | Confidentiality + Integrity + Authentication | Encrypts payload, optional header auth |
| **AH** (Authentication Header) | Integrity + Authentication only | Authenticates entire packet including IP header |
| **IKE** (Internet Key Exchange) | Key Management | Negotiates security associations |

### IPSEC Modes:

1. **Transport Mode**: Encrypts only the payload (data portion) of each packet
   - Used for end-to-end communication between hosts
   - Original IP header remains intact

2. **Tunnel Mode**: Encrypts the entire original IP packet
   - Used for VPNs between gateways
   - New IP header is added

## ESP (Encapsulating Security Payload)

ESP provides:
- **Confidentiality** through encryption (e.g., AES-256)
- **Data integrity** through hash-based authentication
- **Anti-replay protection** using sequence numbers

### ESP Packet Structure:

```
+----------------+------------------+------------------+
| ESP Header     | Encrypted        | ESP Trailer     |
| (SPI + Seq#)   | Payload + Padding| + Auth Data     |
+----------------+------------------+------------------+
```

## AH (Authentication Header)

AH provides:
- **Data integrity** for the entire IP packet
- **Authentication** of the source
- **Anti-replay protection**

⚠️ **Note**: AH does NOT provide encryption (no confidentiality)

In [1]:
# Demonstration: Simulating IPSEC Concepts
# This is a conceptual demonstration - real IPSEC operates at the kernel/network level

import hashlib
import os
from dataclasses import dataclass
from typing import Optional

@dataclass
class IPPacket:
    """Simplified IP Packet representation"""
    source_ip: str
    dest_ip: str
    payload: bytes

    def __repr__(self):
        return f"IP[{self.source_ip} -> {self.dest_ip}] Payload: {self.payload[:30]}..."

@dataclass
class ESPPacket:
    """Simulated ESP packet structure"""
    spi: int                    # Security Parameters Index
    sequence_number: int        # Anti-replay protection
    encrypted_payload: bytes    # The encrypted data
    auth_data: bytes           # Integrity check value (ICV)

@dataclass
class AHPacket:
    """Simulated AH packet structure"""
    spi: int
    sequence_number: int
    auth_data: bytes           # Covers entire packet including IP header
    original_packet: IPPacket  # Original packet (NOT encrypted)

print("✅ IPSEC packet structures defined")
print("\nKey Difference:")
print("- ESP: Encrypts payload → Provides CONFIDENTIALITY")
print("- AH: Does NOT encrypt → Only provides AUTHENTICATION")

✅ IPSEC packet structures defined

Key Difference:
- ESP: Encrypts payload → Provides CONFIDENTIALITY
- AH: Does NOT encrypt → Only provides AUTHENTICATION


In [2]:
# Simulating ESP encryption process
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import hmac

def simulate_esp_encrypt(packet: IPPacket, encryption_key: bytes, auth_key: bytes, spi: int, seq: int) -> ESPPacket:
    """
    Simulates ESP encryption process:
    1. Encrypt the payload using AES-CBC
    2. Calculate authentication data (ICV) using HMAC-SHA256
    """
    # Generate IV for CBC mode
    iv = os.urandom(16)

    # Pad payload to block size
    padding_length = 16 - (len(packet.payload) % 16)
    padded_payload = packet.payload + bytes([padding_length] * padding_length)

    # Encrypt using AES-CBC
    cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_payload = iv + encryptor.update(padded_payload) + encryptor.finalize()

    # Calculate ICV (authentication data) - covers SPI, Seq#, and encrypted payload
    data_to_auth = spi.to_bytes(4, 'big') + seq.to_bytes(4, 'big') + encrypted_payload
    auth_data = hmac.new(auth_key, data_to_auth, hashlib.sha256).digest()[:12]  # Truncated to 96 bits

    return ESPPacket(
        spi=spi,
        sequence_number=seq,
        encrypted_payload=encrypted_payload,
        auth_data=auth_data
    )

# Demo
original_packet = IPPacket(
    source_ip="192.168.1.100",
    dest_ip="10.0.0.50",
    payload=b"CONFIDENTIAL: Q4 Financial Report - Revenue increased by 25%"
)

# Security keys (in real IPSEC, these come from IKE negotiation)
encryption_key = os.urandom(32)  # AES-256 key
auth_key = os.urandom(32)        # HMAC key

esp_packet = simulate_esp_encrypt(original_packet, encryption_key, auth_key, spi=12345, seq=1)

print("Original Packet:")
print(f"  Payload: {original_packet.payload}")
print(f"\nESP Protected Packet:")
print(f"  SPI: {esp_packet.spi}")
print(f"  Sequence #: {esp_packet.sequence_number}")
print(f"  Encrypted Payload (hex): {esp_packet.encrypted_payload.hex()[:60]}...")
print(f"  Auth Data (ICV): {esp_packet.auth_data.hex()}")
print("\n✅ Payload is now ENCRYPTED - an attacker cannot read it!")

Original Packet:
  Payload: b'CONFIDENTIAL: Q4 Financial Report - Revenue increased by 25%'

ESP Protected Packet:
  SPI: 12345
  Sequence #: 1
  Encrypted Payload (hex): 438f1eb3efbc8bb84fc36d87f3d481472d32c6225bdc09ffb7e404d02864...
  Auth Data (ICV): a4c5179615dfe85061349c4f

✅ Payload is now ENCRYPTED - an attacker cannot read it!


In [3]:
# Simulating AH authentication process

def simulate_ah_authenticate(packet: IPPacket, auth_key: bytes, spi: int, seq: int) -> AHPacket:
    """
    Simulates AH authentication process:
    - Creates authentication data covering the ENTIRE packet (including IP header)
    - Does NOT encrypt anything
    """
    # AH authenticates: IP header fields + AH header + original payload
    # (In real AH, mutable IP header fields are zeroed before hashing)
    data_to_auth = (
        packet.source_ip.encode() +
        packet.dest_ip.encode() +
        spi.to_bytes(4, 'big') +
        seq.to_bytes(4, 'big') +
        packet.payload
    )

    auth_data = hmac.new(auth_key, data_to_auth, hashlib.sha256).digest()[:12]

    return AHPacket(
        spi=spi,
        sequence_number=seq,
        auth_data=auth_data,
        original_packet=packet  # Note: payload is NOT encrypted!
    )

ah_packet = simulate_ah_authenticate(original_packet, auth_key, spi=67890, seq=1)

print("AH Protected Packet:")
print(f"  SPI: {ah_packet.spi}")
print(f"  Sequence #: {ah_packet.sequence_number}")
print(f"  Auth Data: {ah_packet.auth_data.hex()}")
print(f"  Original Payload: {ah_packet.original_packet.payload}")
print("\n⚠️ Notice: The payload is STILL READABLE!")
print("AH provides integrity and authentication, but NOT confidentiality.")

AH Protected Packet:
  SPI: 67890
  Sequence #: 1
  Auth Data: b1a95cb9476dd366debb9a31
  Original Payload: b'CONFIDENTIAL: Q4 Financial Report - Revenue increased by 25%'

⚠️ Notice: The payload is STILL READABLE!
AH provides integrity and authentication, but NOT confidentiality.


## IPSEC Implementation Challenges in Dynamic Environments

### Challenge 1: Dynamic IP Addresses
- **Problem**: Mobile workers, DHCP, cloud environments have changing IPs
- **Solution**: Use IKEv2 with MOBIKE extension, or DNS-based endpoint identification

### Challenge 2: NAT Traversal
- **Problem**: NAT modifies IP headers, breaking AH authentication
- **Solution**: NAT-T (NAT Traversal) - encapsulates ESP in UDP port 4500

### Challenge 3: Performance Overhead
- **Problem**: Encryption/decryption adds latency and CPU load
- **Solution**: Hardware acceleration, AES-NI, or dedicated crypto processors

### Challenge 4: Key Management at Scale
- **Problem**: Managing keys for hundreds of endpoints
- **Solution**: PKI integration, certificate-based authentication, centralized management

---
# Part 2: AES Encryption - Maximizing Security

## What is AES?

**AES (Advanced Encryption Standard)** is a symmetric block cipher that:
- Operates on 128-bit blocks
- Supports key sizes of 128, 192, or 256 bits
- Uses substitution-permutation network (SPN) structure
- Is the gold standard for symmetric encryption

### How AES Ensures Confidentiality:
1. **Confusion**: Complex relationship between key and ciphertext (SubBytes, MixColumns)
2. **Diffusion**: Each plaintext bit affects many ciphertext bits (ShiftRows, MixColumns)
3. **Multiple Rounds**: 10/12/14 rounds for 128/192/256-bit keys

In [ ]:
# Install cryptography library if needed
!pip install cryptography -q

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

print("✅ Cryptography library ready!")

## AES Modes of Operation

Since AES only encrypts 128-bit blocks, we need **modes of operation** to handle larger data.

| Mode | Use Case | Parallelizable | Integrity | Notes |
|------|----------|----------------|-----------|-------|
| **ECB** | ❌ Never use | Yes | No | Reveals patterns |
| **CBC** | File encryption, disk encryption | Decrypt only | No | Most common |
| **CTR** | Streaming, random access | Yes | No | Fast, parallelizable |
| **GCM** | Network protocols, TLS | Yes | **Yes** | Authenticated encryption |

In [5]:
# AES-ECB: Why it's DANGEROUS (for educational purposes only!)

from cryptography.hazmat.primitives import padding

def aes_ecb_encrypt(plaintext: bytes, key: bytes) -> bytes:
    """ECB mode - DO NOT USE IN PRODUCTION"""
    # Pad to block size
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext) + padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    return encryptor.update(padded_data) + encryptor.finalize()

# Demonstrate the ECB penguin problem
key = os.urandom(32)  # 256-bit key

# Repeated patterns in plaintext
block1 = b"SAME_PATTERN_123"  # Exactly 16 bytes
block2 = b"DIFF_CONTENT_XYZ"
block3 = b"SAME_PATTERN_123"  # Same as block1!

plaintext = block1 + block2 + block3
ciphertext = aes_ecb_encrypt(plaintext, key)

print("ECB Mode Vulnerability Demonstration:")
print(f"\nPlaintext blocks:")
print(f"  Block 1: {block1}")
print(f"  Block 2: {block2}")
print(f"  Block 3: {block3}")
print(f"\nCiphertext blocks (hex):")
print(f"  Block 1: {ciphertext[0:16].hex()}")
print(f"  Block 2: {ciphertext[16:32].hex()}")
print(f"  Block 3: {ciphertext[32:48].hex()}")
print(f"\n⚠️ Notice: Block 1 and Block 3 ciphertext are IDENTICAL!")
print("This reveals patterns in your data - ECB should NEVER be used!")

ECB Mode Vulnerability Demonstration:

Plaintext blocks:
  Block 1: b'SAME_PATTERN_123'
  Block 2: b'DIFF_CONTENT_XYZ'
  Block 3: b'SAME_PATTERN_123'

Ciphertext blocks (hex):
  Block 1: b034a24aafa42b8cb47e3e21672f1eb2
  Block 2: df6a31ddc7ea7a1dbd76657d9d0d26f9
  Block 3: b034a24aafa42b8cb47e3e21672f1eb2

⚠️ Notice: Block 1 and Block 3 ciphertext are IDENTICAL!
This reveals patterns in your data - ECB should NEVER be used!


In [6]:
# AES-CBC: Cipher Block Chaining - The Standard Choice

def aes_cbc_encrypt(plaintext: bytes, key: bytes) -> tuple[bytes, bytes]:
    """
    AES-CBC encryption
    Returns: (iv, ciphertext)
    """
    iv = os.urandom(16)  # Random IV for each encryption

    # Pad to block size
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext) + padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return iv, ciphertext

def aes_cbc_decrypt(iv: bytes, ciphertext: bytes, key: bytes) -> bytes:
    """
    AES-CBC decryption
    """
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(128).unpadder()
    return unpadder.update(padded_data) + unpadder.finalize()

# Example: Encrypting customer data
customer_data = b"Customer: John Doe | Card: 4111-1111-1111-1111 | CVV: 123"
key = os.urandom(32)  # 256-bit key

iv, ciphertext = aes_cbc_encrypt(customer_data, key)
decrypted = aes_cbc_decrypt(iv, ciphertext, key)

print("AES-CBC Encryption Example:")
print(f"\nOriginal data: {customer_data}")
print(f"\nIV (must be stored with ciphertext): {iv.hex()}")
print(f"Ciphertext: {ciphertext.hex()}")
print(f"\nDecrypted: {decrypted}")
print(f"\n✅ Decryption successful: {customer_data == decrypted}")

# Show that same plaintext produces different ciphertext each time
print("\n--- Demonstrating IV importance ---")
iv2, ciphertext2 = aes_cbc_encrypt(customer_data, key)
print(f"Same plaintext, different IV:")
print(f"  Ciphertext 1: {ciphertext.hex()[:40]}...")
print(f"  Ciphertext 2: {ciphertext2.hex()[:40]}...")
print(f"\n✅ Different ciphertexts! Random IV prevents pattern detection.")

AES-CBC Encryption Example:

Original data: b'Customer: John Doe | Card: 4111-1111-1111-1111 | CVV: 123'

IV (must be stored with ciphertext): 25acd0ec40f581f5837b88c846c53d4c
Ciphertext: d296aebdaa509026a7ed2aece620e64eee7589d147f9fe0dbfcc58f6e130870e8b7a187db086cb52f586353864fb99abe166238a5006431deacad63989ff1207

Decrypted: b'Customer: John Doe | Card: 4111-1111-1111-1111 | CVV: 123'

✅ Decryption successful: True

--- Demonstrating IV importance ---
Same plaintext, different IV:
  Ciphertext 1: d296aebdaa509026a7ed2aece620e64eee7589d1...
  Ciphertext 2: 858d506264c16ad8660747b91cf048ee76a90a2b...

✅ Different ciphertexts! Random IV prevents pattern detection.


In [7]:
# AES-GCM: Galois/Counter Mode - The Modern Choice
# Provides both encryption AND authentication (AEAD - Authenticated Encryption with Associated Data)

from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def aes_gcm_encrypt(plaintext: bytes, key: bytes, associated_data: bytes = None) -> tuple[bytes, bytes]:
    """
    AES-GCM encryption with authentication
    Returns: (nonce, ciphertext_with_tag)

    Associated data is authenticated but NOT encrypted (e.g., packet headers)
    """
    nonce = os.urandom(12)  # 96-bit nonce is standard for GCM
    aesgcm = AESGCM(key)
    ciphertext = aesgcm.encrypt(nonce, plaintext, associated_data)
    return nonce, ciphertext

def aes_gcm_decrypt(nonce: bytes, ciphertext: bytes, key: bytes, associated_data: bytes = None) -> bytes:
    """
    AES-GCM decryption with authentication verification
    Raises InvalidTag if authentication fails (data was tampered with)
    """
    aesgcm = AESGCM(key)
    return aesgcm.decrypt(nonce, ciphertext, associated_data)

# Example: Secure API request
api_request = b'{"action": "transfer", "amount": 10000, "to_account": "12345678"}'
request_headers = b"Request-ID: abc123; Timestamp: 2025-01-01T12:00:00Z"  # Associated data

key = AESGCM.generate_key(bit_length=256)

nonce, ciphertext = aes_gcm_encrypt(api_request, key, request_headers)

print("AES-GCM Authenticated Encryption Example:")
print(f"\nPlaintext (API request): {api_request}")
print(f"Associated data (headers): {request_headers}")
print(f"\nNonce: {nonce.hex()}")
print(f"Ciphertext + Auth Tag: {ciphertext.hex()}")
print(f"  (Last 16 bytes are the authentication tag)")

# Successful decryption
decrypted = aes_gcm_decrypt(nonce, ciphertext, key, request_headers)
print(f"\nDecrypted: {decrypted}")
print("✅ Authentication and decryption successful!")

AES-GCM Authenticated Encryption Example:

Plaintext (API request): b'{"action": "transfer", "amount": 10000, "to_account": "12345678"}'
Associated data (headers): b'Request-ID: abc123; Timestamp: 2025-01-01T12:00:00Z'

Nonce: edde1d63d6c373ad96107cd7
Ciphertext + Auth Tag: e55eed4c5465de395ac4641ba4bfd1d48d2531bff97bfc08f8683ae2e0d653220bc3be315f45a468abe01cd972b63f1053266838aeaf4f7f719dc7eabfb9b3126ae6a28a868a93cf6f6ae03d56413b1721
  (Last 16 bytes are the authentication tag)

Decrypted: b'{"action": "transfer", "amount": 10000, "to_account": "12345678"}'
✅ Authentication and decryption successful!


In [8]:
# Demonstrating GCM's tamper detection

print("--- Demonstrating GCM Tamper Detection ---\n")

# Try to tamper with the ciphertext
tampered_ciphertext = bytearray(ciphertext)
tampered_ciphertext[5] ^= 0xFF  # Flip some bits
tampered_ciphertext = bytes(tampered_ciphertext)

try:
    decrypted = aes_gcm_decrypt(nonce, tampered_ciphertext, key, request_headers)
    print("Decryption succeeded (this shouldn't happen!)")
except Exception as e:
    print(f"❌ Tamper detected! Decryption failed with: {type(e).__name__}")
    print("   The ciphertext was modified - authentication failed.")

# Try to modify associated data
print("\n--- Tampering with associated data ---")
fake_headers = b"Request-ID: HACKED; Timestamp: 2025-01-01T12:00:00Z"

try:
    decrypted = aes_gcm_decrypt(nonce, ciphertext, key, fake_headers)
    print("Decryption succeeded (this shouldn't happen!)")
except Exception as e:
    print(f"❌ Tamper detected! Decryption failed with: {type(e).__name__}")
    print("   The associated data was modified - authentication failed.")

print("\n✅ GCM provides BOTH confidentiality AND integrity protection!")

--- Demonstrating GCM Tamper Detection ---

❌ Tamper detected! Decryption failed with: InvalidTag
   The ciphertext was modified - authentication failed.

--- Tampering with associated data ---
❌ Tamper detected! Decryption failed with: InvalidTag
   The associated data was modified - authentication failed.

✅ GCM provides BOTH confidentiality AND integrity protection!


## AES Key Management Best Practices

The security of AES depends entirely on protecting the keys!

### Key Storage Considerations:

1. **Never hardcode keys** in source code
2. **Use environment variables** or secure vaults (AWS KMS, HashiCorp Vault, Azure Key Vault)
3. **Key derivation** - derive keys from passwords using PBKDF2, scrypt, or Argon2
4. **Hardware Security Modules (HSM)** for high-security applications
5. **Key rotation** - regularly change keys to limit exposure

In [9]:
# Key Derivation from Password using PBKDF2

from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import base64

def derive_key_from_password(password: str, salt: bytes = None) -> tuple[bytes, bytes]:
    """
    Derive a secure AES key from a password using PBKDF2
    Returns: (salt, key)

    The salt must be stored alongside encrypted data for decryption!
    """
    if salt is None:
        salt = os.urandom(16)  # Random salt for each key

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,         # 256-bit key
        salt=salt,
        iterations=480000,  # OWASP recommended minimum (2023)
        backend=default_backend()
    )

    key = kdf.derive(password.encode())
    return salt, key

# Example: User wants to encrypt their files with a password
user_password = "MySecureP@ssw0rd123!"

salt, derived_key = derive_key_from_password(user_password)

print("Key Derivation from Password:")
print(f"\nPassword: {user_password}")
print(f"Salt: {salt.hex()}")
print(f"Derived Key: {derived_key.hex()}")
print(f"\n⚠️ Important: Store the salt with your encrypted data!")
print("   Without it, you cannot recreate the key from the password.")

# Verify same password + salt = same key
_, derived_key2 = derive_key_from_password(user_password, salt)
print(f"\n✅ Same password + salt produces same key: {derived_key == derived_key2}")

Key Derivation from Password:

Password: MySecureP@ssw0rd123!
Salt: 5735295f81eb20e2b05b7f0219f2c104
Derived Key: db163f0c6720fc3969924834df253806493ec7ec9cae5576296ec3786fcfcc45

⚠️ Important: Store the salt with your encrypted data!
   Without it, you cannot recreate the key from the password.

✅ Same password + salt produces same key: True


In [10]:
# Complete Example: Secure Database Field Encryption

class SecureFieldEncryptor:
    """
    Example class for encrypting sensitive database fields.
    Uses AES-GCM for authenticated encryption.
    """

    def __init__(self, master_key: bytes):
        self.master_key = master_key

    def encrypt_field(self, field_name: str, value: str) -> dict:
        """
        Encrypt a field value with the field name as associated data.
        Returns a dict containing all info needed for decryption.
        """
        nonce = os.urandom(12)
        aesgcm = AESGCM(self.master_key)

        # Field name is associated data - authenticated but not encrypted
        ciphertext = aesgcm.encrypt(
            nonce,
            value.encode(),
            field_name.encode()
        )

        return {
            'field': field_name,
            'nonce': base64.b64encode(nonce).decode(),
            'ciphertext': base64.b64encode(ciphertext).decode()
        }

    def decrypt_field(self, encrypted_data: dict) -> str:
        """
        Decrypt a field value and verify it hasn't been tampered with.
        """
        nonce = base64.b64decode(encrypted_data['nonce'])
        ciphertext = base64.b64decode(encrypted_data['ciphertext'])

        aesgcm = AESGCM(self.master_key)
        plaintext = aesgcm.decrypt(
            nonce,
            ciphertext,
            encrypted_data['field'].encode()
        )

        return plaintext.decode()

# Usage example
master_key = AESGCM.generate_key(bit_length=256)
encryptor = SecureFieldEncryptor(master_key)

# Encrypt sensitive customer fields
encrypted_ssn = encryptor.encrypt_field('social_security_number', '123-45-6789')
encrypted_card = encryptor.encrypt_field('credit_card', '4111-1111-1111-1111')

print("Encrypted Database Fields:")
print(f"\nSSN: {encrypted_ssn}")
print(f"\nCard: {encrypted_card}")

# Decrypt
print(f"\nDecrypted SSN: {encryptor.decrypt_field(encrypted_ssn)}")
print(f"Decrypted Card: {encryptor.decrypt_field(encrypted_card)}")

print("\n✅ Fields encrypted and decrypted successfully!")

Encrypted Database Fields:

SSN: {'field': 'social_security_number', 'nonce': 'z94/eR69AUW7/IQD', 'ciphertext': 'YKp+Aalkzr8PzN+k9a553Y+uNDClfmNUNla9'}

Card: {'field': 'credit_card', 'nonce': 'XLvulr1mcGxD+RaF', 'ciphertext': '4rvUYNCHk+LV+ERqomIV/25TKz/TZFiuO3HML1zgt+D+3Xg='}

Decrypted SSN: 123-45-6789
Decrypted Card: 4111-1111-1111-1111

✅ Fields encrypted and decrypted successfully!


---
# Part 3: SHA Hashing for Data Integrity

## What is SHA?

**SHA (Secure Hash Algorithm)** is a family of cryptographic hash functions that:
- Produce a fixed-size output (digest/hash) from arbitrary input
- Are **one-way** (cannot reverse the hash to get the original)
- Are **collision-resistant** (hard to find two inputs with the same hash)

### SHA's Role in Data Integrity:

1. **File Verification**: Ensure downloaded files haven't been modified
2. **Digital Signatures**: Hash the message before signing
3. **Password Storage**: Store hash, not plaintext passwords
4. **Blockchain**: Chain blocks together with hashes

In [11]:
import hashlib

# Basic SHA hashing demonstration
message = b"This is a test message for hashing."

print("SHA Hash Comparison:")
print(f"\nOriginal message: {message}")
print(f"Message length: {len(message)} bytes\n")

# SHA-1 (160 bits = 40 hex chars)
sha1_hash = hashlib.sha1(message).hexdigest()
print(f"SHA-1   ({len(sha1_hash)*4} bits): {sha1_hash}")

# SHA-256 (256 bits = 64 hex chars)
sha256_hash = hashlib.sha256(message).hexdigest()
print(f"SHA-256 ({len(sha256_hash)*4} bits): {sha256_hash}")

# SHA-512 (512 bits = 128 hex chars)
sha512_hash = hashlib.sha512(message).hexdigest()
print(f"SHA-512 ({len(sha512_hash)*4} bits): {sha512_hash}")

# SHA-3 (Keccak-based, different internal structure)
sha3_256_hash = hashlib.sha3_256(message).hexdigest()
print(f"SHA3-256 ({len(sha3_256_hash)*4} bits): {sha3_256_hash}")

SHA Hash Comparison:

Original message: b'This is a test message for hashing.'
Message length: 35 bytes

SHA-1   (160 bits): 919030338f1cded6fe8d42352639847ff885f474
SHA-256 (256 bits): e79fdceefaa6017ed885a2285bdc4e199dac34ae5eef0eea47ef99d469b0aa52
SHA-512 (512 bits): 0d4f6715b554646ab45039340282eee0b94de80a86ac539f6b7123ce1c0b2b4584130eda81dfec8495146e26e5f0453072e48100f98f34202314325f522d34b8
SHA3-256 (256 bits): ad429aa7a07fcad941481c4600b424a3c81ca1ccd3c99e1487d12d7f5d3dfef9


In [12]:
# Demonstrating the Avalanche Effect
# A small change in input causes a massive change in output

message1 = b"Hello, World!"
message2 = b"Hello, World."  # Only one character different!

hash1 = hashlib.sha256(message1).hexdigest()
hash2 = hashlib.sha256(message2).hexdigest()

print("Avalanche Effect Demonstration:")
print(f"\nMessage 1: {message1}")
print(f"Message 2: {message2}")
print(f"\nDifference: Just one character ('!' vs '.')")
print(f"\nSHA-256 of Message 1: {hash1}")
print(f"SHA-256 of Message 2: {hash2}")

# Count differing characters
diff_count = sum(1 for a, b in zip(hash1, hash2) if a != b)
print(f"\nDiffering characters: {diff_count} out of 64 ({diff_count/64*100:.1f}%)")
print("\n✅ This unpredictability makes it impossible to 'guess' the original input!")

Avalanche Effect Demonstration:

Message 1: b'Hello, World!'
Message 2: b'Hello, World.'

Difference: Just one character ('!' vs '.')

SHA-256 of Message 1: dffd6021bb2bd5b0af676290809ec3a53191dd81c7f70a4b28688a362182986f
SHA-256 of Message 2: 02b5dcd5f0ef1a39cffec5f8b625ec20bffcf918e4efd3f54babec4eae03b347

Differing characters: 63 out of 64 (98.4%)

✅ This unpredictability makes it impossible to 'guess' the original input!


## SHA Version Comparison

| Algorithm | Output Size | Status | Use Case |
|-----------|-------------|--------|----------|
| **SHA-1** | 160 bits | ❌ DEPRECATED | Legacy only (avoid!) |
| **SHA-256** | 256 bits | ✅ Secure | General purpose, TLS, Bitcoin |
| **SHA-384** | 384 bits | ✅ Secure | High security applications |
| **SHA-512** | 512 bits | ✅ Secure | 64-bit systems, digital signatures |
| **SHA-3** | Variable | ✅ Secure | Future-proofing, backup to SHA-2 |

In [13]:
# SHA-1 Vulnerability: Collision Attack
# Google's SHAttered attack (2017) demonstrated practical SHA-1 collisions

print("SHA-1 Vulnerability Explanation:")
print("="*50)
print("""
In 2017, Google and CWI demonstrated the first practical SHA-1 collision:
- Two different PDF files with the same SHA-1 hash
- Required 9,223,372,036,854,775,808 SHA-1 computations
- Took 110 GPU-years to compute

WHY THIS MATTERS:
1. Attackers can create malicious files that match legitimate file hashes
2. Digital signatures using SHA-1 can be forged
3. Certificate authorities using SHA-1 are vulnerable

COST TO ATTACK:
- In 2017: ~$110,000 using cloud GPU instances
- In 2025: Significantly cheaper due to improved hardware
""")

print("\n⚠️  NEVER use SHA-1 for security-critical applications!")
print("    Use SHA-256 or SHA-3 instead.")

SHA-1 Vulnerability Explanation:

In 2017, Google and CWI demonstrated the first practical SHA-1 collision:
- Two different PDF files with the same SHA-1 hash
- Required 9,223,372,036,854,775,808 SHA-1 computations
- Took 110 GPU-years to compute

WHY THIS MATTERS:
1. Attackers can create malicious files that match legitimate file hashes
2. Digital signatures using SHA-1 can be forged
3. Certificate authorities using SHA-1 are vulnerable

COST TO ATTACK:
- In 2017: ~$110,000 using cloud GPU instances
- In 2025: Significantly cheaper due to improved hardware


⚠️  NEVER use SHA-1 for security-critical applications!
    Use SHA-256 or SHA-3 instead.


In [14]:
# Practical Example: File Integrity Verification for E-commerce Platform

import os
import json
from datetime import datetime

class FileIntegrityChecker:
    """
    File integrity verification system using SHA-256.
    Suitable for an e-commerce platform to verify uploaded files.
    """

    def __init__(self, algorithm='sha256'):
        self.algorithm = algorithm

    def hash_file(self, file_content: bytes) -> str:
        """
        Generate SHA-256 hash of file content.
        Uses chunked reading for large files.
        """
        hasher = hashlib.new(self.algorithm)
        hasher.update(file_content)
        return hasher.hexdigest()

    def hash_large_file(self, filepath: str, chunk_size: int = 8192) -> str:
        """
        Hash a large file by reading in chunks.
        Memory-efficient for files of any size.
        """
        hasher = hashlib.new(self.algorithm)
        with open(filepath, 'rb') as f:
            while chunk := f.read(chunk_size):
                hasher.update(chunk)
        return hasher.hexdigest()

    def create_integrity_record(self, filename: str, content: bytes) -> dict:
        """
        Create an integrity record for audit purposes.
        """
        return {
            'filename': filename,
            'hash_algorithm': self.algorithm.upper(),
            'hash_value': self.hash_file(content),
            'file_size': len(content),
            'timestamp': datetime.now().isoformat(),
        }

    def verify_integrity(self, content: bytes, expected_hash: str) -> bool:
        """
        Verify that content matches the expected hash.
        """
        return self.hash_file(content) == expected_hash

# Simulate e-commerce file upload scenario
checker = FileIntegrityChecker()

# Simulate uploaded product images
product_image = b"\x89PNG\r\n\x1a\n" + os.urandom(1000)  # Simulated PNG
invoice_pdf = b"%PDF-1.4" + os.urandom(2000)  # Simulated PDF

# Create integrity records
image_record = checker.create_integrity_record('product_001.png', product_image)
invoice_record = checker.create_integrity_record('invoice_2025.pdf', invoice_pdf)

print("E-commerce File Integrity Records:")
print("="*50)
print(f"\nProduct Image:")
print(json.dumps(image_record, indent=2))
print(f"\nInvoice PDF:")
print(json.dumps(invoice_record, indent=2))

E-commerce File Integrity Records:

Product Image:
{
  "filename": "product_001.png",
  "hash_algorithm": "SHA256",
  "hash_value": "c228ccf8c3c8b8e769113874b49480d762dd0993e973722d4681ffe1291e1a60",
  "file_size": 1008,
  "timestamp": "2025-11-25T15:27:03.183734"
}

Invoice PDF:
{
  "filename": "invoice_2025.pdf",
  "hash_algorithm": "SHA256",
  "hash_value": "6a63ea565be64de66c60e11cdf8572e054d68efddc923fa63963ac20d796501c",
  "file_size": 2008,
  "timestamp": "2025-11-25T15:27:03.183821"
}


In [15]:
# Demonstrate integrity verification

print("\nIntegrity Verification Test:")
print("="*50)

# Test 1: Verify unmodified file
is_valid = checker.verify_integrity(product_image, image_record['hash_value'])
print(f"\n✅ Unmodified file verification: {'PASSED' if is_valid else 'FAILED'}")

# Test 2: Detect tampered file
tampered_image = bytearray(product_image)
tampered_image[100] = 0xFF  # Modify one byte
tampered_image = bytes(tampered_image)

is_valid = checker.verify_integrity(tampered_image, image_record['hash_value'])
print(f"❌ Tampered file verification: {'PASSED' if is_valid else 'FAILED (tampering detected!)'}")

print("\n" + "="*50)
print("Hash comparison:")
print(f"  Original: {checker.hash_file(product_image)}")
print(f"  Tampered: {checker.hash_file(tampered_image)}")
print("\n✅ Even a single bit change produces a completely different hash!")


Integrity Verification Test:

✅ Unmodified file verification: PASSED
❌ Tampered file verification: FAILED (tampering detected!)

Hash comparison:
  Original: c228ccf8c3c8b8e769113874b49480d762dd0993e973722d4681ffe1291e1a60
  Tampered: 4060bbbdc0f5dc343c306399c2190b3b069fcce1109d047b98f2c050d63002cf

✅ Even a single bit change produces a completely different hash!


In [16]:
# Comparing SHA-2 vs SHA-3

import time

# Generate test data
test_data = os.urandom(1_000_000)  # 1 MB of random data

print("SHA-2 vs SHA-3 Comparison:")
print("="*50)

# SHA-256 (SHA-2 family)
start = time.time()
sha256_result = hashlib.sha256(test_data).hexdigest()
sha256_time = time.time() - start

# SHA3-256 (SHA-3 family)
start = time.time()
sha3_result = hashlib.sha3_256(test_data).hexdigest()
sha3_time = time.time() - start

print(f"\nHashing 1 MB of data:")
print(f"\nSHA-256:")
print(f"  Hash: {sha256_result[:32]}...")
print(f"  Time: {sha256_time*1000:.2f} ms")

print(f"\nSHA3-256:")
print(f"  Hash: {sha3_result[:32]}...")
print(f"  Time: {sha3_time*1000:.2f} ms")

print("\n" + "="*50)
print("""
KEY DIFFERENCES:

SHA-2 (SHA-256, SHA-512):
  ✓ Based on Merkle-Damgård construction
  ✓ Faster on most hardware (hardware acceleration common)
  ✓ Widely deployed, battle-tested
  ✓ Bitcoin, TLS, code signing

SHA-3 (Keccak):
  ✓ Based on sponge construction (completely different design)
  ✓ Immune to length extension attacks
  ✓ Provides a backup if SHA-2 is broken
  ✓ Newer, less widely deployed
  ✓ Variable output length (SHAKE variants)
""")

SHA-2 vs SHA-3 Comparison:

Hashing 1 MB of data:

SHA-256:
  Hash: a1ca6a30f768383d24726bb64cd8ba9e...
  Time: 1.22 ms

SHA3-256:
  Hash: 4e4224d5fd07718f1d6e431f51ee52a3...
  Time: 2.95 ms


KEY DIFFERENCES:

SHA-2 (SHA-256, SHA-512):
  ✓ Based on Merkle-Damgård construction
  ✓ Faster on most hardware (hardware acceleration common)
  ✓ Widely deployed, battle-tested
  ✓ Bitcoin, TLS, code signing

SHA-3 (Keccak):
  ✓ Based on sponge construction (completely different design)
  ✓ Immune to length extension attacks
  ✓ Provides a backup if SHA-2 is broken
  ✓ Newer, less widely deployed
  ✓ Variable output length (SHAKE variants)



In [18]:
# Complete Solution: File Upload Verification System

import hashlib
import os
from datetime import datetime
from typing import Tuple, Dict, List
import json

class SecureFileUploadSystem:
    """
    Complete file upload verification system for an e-commerce platform.
    Uses SHA-256 for file integrity verification.
    """

    ALLOWED_EXTENSIONS = {'.pdf', '.png', '.jpg', '.jpeg', '.docx'}
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10 MB

    def __init__(self):
        self.file_registry: Dict[str, dict] = {}

    def calculate_hash(self, content: bytes, algorithm: str = 'sha256') -> str:
        """
        Calculate cryptographic hash of file content.
        """
        return hashlib.new(algorithm, content).hexdigest()

    def upload_file(self, filename: str, content: bytes) -> Tuple[bool, str, dict]:
        """
        Process a file upload with integrity verification.

        Returns:
            Tuple of (success, message, file_record)
        """
        # Validate file extension
        ext = os.path.splitext(filename)[1].lower()
        if ext not in self.ALLOWED_EXTENSIONS:
            return False, f"File type {ext} not allowed", {}

        # Validate file size
        if len(content) > self.MAX_FILE_SIZE:
            return False, "File exceeds maximum size of 10 MB", {}

        # Generate unique file ID and calculate hash
        file_id = hashlib.sha256(
            f"{filename}{datetime.now().isoformat()}{os.urandom(8).hex()}".encode()
        ).hexdigest()[:16]

        file_hash = self.calculate_hash(content)

        # Create file record
        record = {
            'file_id': file_id,
            'original_filename': filename,
            'file_size': len(content),
            'sha256_hash': file_hash,
            'upload_timestamp': datetime.now().isoformat(),
            'verified': True
        }

        self.file_registry[file_id] = record

        return True, "File uploaded successfully", record

    def verify_file(self, file_id: str, content: bytes) -> Tuple[bool, str]:
        """
        Verify file integrity against stored hash.
        """
        if file_id not in self.file_registry:
            return False, "File not found in registry"

        stored_hash = self.file_registry[file_id]['sha256_hash']
        current_hash = self.calculate_hash(content)

        if stored_hash == current_hash:
            return True, "✅ File integrity verified - no modifications detected"
        else:
            return False, "❌ INTEGRITY FAILURE - File has been modified!"

    def get_audit_log(self) -> str:
        """
        Generate audit log of all files.
        """
        return json.dumps(list(self.file_registry.values()), indent=2)

# Demo the system
upload_system = SecureFileUploadSystem()

# Simulate uploading various files
files = [
    ('product_photo.jpg', os.urandom(5000)),
    ('invoice_2025.pdf', b'%PDF-1.4' + os.urandom(8000)),
    ('contract.docx', os.urandom(15000)),
]

print("Secure File Upload System Demo:")
print("="*60)

uploaded_files = []
for filename, content in files:
    success, message, record = upload_system.upload_file(filename, content)
    if success:
        uploaded_files.append((record['file_id'], content))
        print(f"\n✅ {filename}")
        print(f"   ID: {record['file_id']}")
        print(f"   Hash: {record['sha256_hash'][:32]}...")

print("\n" + "="*60)
print("\nVerification Tests:")

# Verify unmodified file
file_id, original_content = uploaded_files[0]
success, message = upload_system.verify_file(file_id, original_content)
print(f"\nOriginal file: {message}")

# Try to verify tampered file
tampered = bytearray(original_content)
tampered[100] = (tampered[100] + 1) % 256
success, message = upload_system.verify_file(file_id, bytes(tampered))
print(f"Tampered file: {message}")

Secure File Upload System Demo:

✅ product_photo.jpg
   ID: 6392017a95d9c546
   Hash: ec6d6e61e78b42ee68f3fadfb0dfbf90...

✅ invoice_2025.pdf
   ID: 11f8f1afd8f408e7
   Hash: 4e3d481bfe316fd25b63c551095e3366...

✅ contract.docx
   ID: a550641b282bcf5f
   Hash: bf502d15557e2ee2d3cd24ed06675cd0...


Verification Tests:

Original file: ✅ File integrity verified - no modifications detected
Tampered file: ❌ INTEGRITY FAILURE - File has been modified!


---
# Summary and Key Takeaways

## IPSEC
- **ESP** provides confidentiality (encryption) + integrity + authentication
- **AH** provides integrity + authentication only (no encryption)
- Key challenges: NAT traversal, dynamic IPs, key management at scale

## AES
- **AES-GCM** is the recommended mode (encryption + authentication)
- **AES-CBC** is acceptable but requires separate HMAC for integrity
- **Never use ECB** - reveals patterns in data
- Key management is critical: use KMS, HSM, or secure key derivation

## SHA
- **SHA-1 is DEPRECATED** - vulnerable to collision attacks
- **SHA-256** is the current standard for most applications
- **SHA-3** provides a backup if SHA-2 is ever broken
- Use hashing for file integrity, password storage, digital signatures

---

## References for Further Reading

1. RFC 4303 - IP Encapsulating Security Payload (ESP)
2. RFC 4302 - IP Authentication Header (AH)
3. NIST FIPS 197 - Advanced Encryption Standard (AES)
4. NIST FIPS 180-4 - Secure Hash Standard (SHA)
5. NIST FIPS 202 - SHA-3 Standard
6. OWASP Cryptographic Storage Cheat Sheet

---
# Practice Exercises

Try these exercises to reinforce your understanding:

1. **AES Exercise**: Modify the `SecureFieldEncryptor` class to support multiple encryption modes (CBC and GCM). Allow the user to choose which mode to use.

2. **SHA Exercise**: Create a function that compares the performance of SHA-256, SHA-512, and SHA3-256 across different file sizes (1KB, 1MB, 100MB).

3. **Integration Exercise**: Combine AES-GCM encryption with SHA-256 hashing to create a secure file storage system that:
   - Encrypts file contents
   - Creates a hash of the encrypted file for integrity
   - Stores metadata including file hash, encryption nonce, and timestamp